# Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp '/content/drive/My Drive/GIZ Zindi/Train.csv' .
!cp '/content/drive/My Drive/GIZ Zindi/SampleSubmission.csv' .

In [3]:
!cp '/content/drive/My Drive/GIZ Zindi/AdditionalUtterances.zip' AdditionalUtterances.zip

In [4]:
!unzip -q AdditionalUtterances.zip

In [5]:
# Copy the files in and unzip
!cp '/content/drive/My Drive/GIZ Zindi/audio_files.zip' audio_files.zip
!unzip -q audio_files.zip

In [6]:
!cp "/content/drive/My Drive/GIZ Zindi/nlp_keywords_29Oct2020.zip" nlp_keywords_29Oct2020.zip
!unzip -q nlp_keywords_29Oct2020.zip

In [7]:
!pip -q install efficientnet_pytorch

In [8]:
!pip install -q python_speech_features

In [9]:
!pip -q install albumentations --upgrade

     |████████████████████████████████| 81kB 6.9MB/s 
     |████████████████████████████████| 36.7MB 88kB/s 
     |████████████████████████████████| 952kB 47.0MB/s 


In [10]:
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from torchvision import datasets, models
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch
import torchvision.models as models
from efficientnet_pytorch import EfficientNet
from torch.optim.lr_scheduler import MultiStepLR
from torch.optim.lr_scheduler import OneCycleLR
import pandas as pd 
import numpy as np
import sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score
from tqdm.notebook import tqdm as tqdm 
from sklearn.model_selection import train_test_split
import librosa
import librosa.display as display
import python_speech_features as psf
from matplotlib import pyplot as plt
import numpy as np
import albumentations
from torch.nn import Module,Sequential
import gc
import cv2
import multiprocessing as mp 
from multiprocessing import Pool
from albumentations.augmentations.transforms import Lambda
import IPython.display as ipd 

In [11]:
N_WORKERS = mp.cpu_count()
LOAD_TRAIN_DATA = None
LOAD_TEST_DATA = None

In [12]:
import random
import numpy as np
SEED_VAL  = 1000
# Set the seed value all over the place to make this reproducible.
def seed_all(SEED = SEED_VAL):
  random.seed(SEED)
  np.random.seed(SEED)
  torch.manual_seed(SEED)
  torch.cuda.manual_seed_all(SEED)
  os.environ['PYTHONHASHSEED'] = str(SEED)
  torch.backends.cudnn.deterministic = True

# DataLoader

In [13]:
class conf:
    sampling_rate = 44100
    duration = 3 # sec
    hop_length = 200*duration # to make time steps 128
    fmin = 20
    fmax = sampling_rate // 2
    n_mels = 128
    n_fft = n_mels * 20
    padmode = 'constant'
    samples = sampling_rate * duration
def get_default_conf():
    return conf
conf = get_default_conf()

In [14]:
def melspectogram_dB(file_path, cst=3, top_db=80.):
  row_sound, sr = librosa.load(file_path,sr=conf.sampling_rate)
  sound = np.zeros((cst*sr,))
  if row_sound.shape[0] < cst*sr:
    sound[:row_sound.shape[0]] = row_sound[:]
  else:
    sound[:] = row_sound[:cst*sr]

  spec = librosa.feature.melspectrogram(sound, 
                                    sr=conf.sampling_rate,
                                    n_mels=conf.n_mels,
                                    hop_length=conf.hop_length,
                                    n_fft=conf.n_fft,
                                    fmin=conf.fmin,
                                    fmax=conf.fmax)
  spec_db = librosa.power_to_db(spec)
  spec_db = spec_db.astype(np.float32)

  return spec_db

def spec_to_image(spec, eps=1e-6):
  mean = spec.mean()
  std = spec.std()
  spec_norm = (spec - mean) / (std + eps)
  spec_min, spec_max = spec_norm.min(), spec_norm.max()
  spec_img = 255 * (spec_norm - spec_min) / (spec_max - spec_min)
  
  return spec_img.astype(np.uint8)
def preprocess_audio(audio_path):
  spec = melspectogram_dB(audio_path)
  spec = spec_to_image(spec)
  return spec

In [15]:
def get_data(df,mode='train'):
  """
  :param: df: dataframe of train or test
  :return: images_list: spec images of all the data
  :return: label_list : label list of all the data
  """
  audio_paths = df.fn.values
  images_list = []
  with mp.Pool(N_WORKERS) as pool:
    images_list = pool.map(preprocess_audio,tqdm(audio_paths))

  if mode == 'train':
    label_list = df.label.values
    return images_list,label_list
  else:
    return images_list

In [16]:
class ImageDataset(Dataset):
    def __init__(self, images_list,labels_list=None,transform=None):
        self.images_list = images_list
        self.transform = transform
        self.labels_list = labels_list

    def __getitem__(self, index):
      
      spec = self.images_list[index]
      if self.transform is not None:
        spec = self.transform(image=spec)
        spec = spec['image']
        
      if self.labels_list is not None:
          label = self.labels_list[index]
          return {'image' : torch.tensor(spec,dtype=torch.float), 
              'label' : torch.tensor(label,dtype = torch.long) }
          
      return {'image' : torch.tensor(spec,dtype=torch.float), }

    def __len__(self):
        return len(self.images_list)

# Models and train functions

In [26]:
class Net(nn.Module):
    def __init__(self,name):
        super(Net, self).__init__()
        self.name  = name
        #self.convert_3_channels = nn.Conv2d(1,3,2,padding=1)
        if name == 'b0':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b0')
          self.arch._fc = nn.Linear(in_features=1280, out_features=193, bias=True)
        elif name == 'b1':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b1')
          self.arch._fc = nn.Linear(in_features=1280, out_features=193, bias=True)
        elif name == 'b2':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b2')
          self.arch._fc = nn.Linear(in_features=1408, out_features=193, bias=True)
        elif name =='b3':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b3')
          self.arch._fc = nn.Linear(in_features=1536, out_features=193, bias=True)
        elif name =='b4':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b4')
          self.arch._fc = nn.Linear(in_features=1792, out_features=193, bias=True,)
        elif name =='b5':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b5')
          self.arch._fc = nn.Linear(in_features=2048, out_features=193, bias=True)
        elif name =='b6':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b6')
          self.arch._fc = nn.Linear(in_features=2304, out_features=193, bias=True)
        elif name =='b7':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b7')
          self.arch._fc = nn.Linear(in_features=2560, out_features=193, bias=True)
        elif name == 'densenet121':
          self.arch = models.densenet121(pretrained=True)
          num_ftrs = self.arch.classifier.in_features
          self.arch.classifier = nn.Linear(num_ftrs,193,bias=True)
        elif name == 'densenet169':
          self.arch = models.densenet169(pretrained=True)
          num_ftrs = self.arch.classifier.in_features
          self.arch.classifier = nn.Linear(num_ftrs,193,bias=True)
        elif name == 'densenet201':
          self.arch = models.densenet201(pretrained=True)
          num_ftrs = self.arch.classifier.in_features
          self.arch.classifier = nn.Linear(num_ftrs,193,bias=True)
        elif name == 'resnet50':
          self.arch = models.resnet50(pretrained=True)
          num_ftrs = self.arch.fc.in_features
          self.arch.fc = nn.Linear(num_ftrs,193,bias=True)
        elif name == 'resnet101':
          self.arch = models.resnet101(pretrained=True)
          num_ftrs = self.arch.fc.in_features
          self.arch.fc = nn.Linear(num_ftrs,193,bias=True)
        elif name == 'resnet152':
          self.arch = models.resnet152(pretrained=True)
          num_ftrs = self.arch.fc.in_features
          self.arch.fc = nn.Linear(num_ftrs,193,bias=True)
        elif name == 'resnet18':
          self.arch = models.resnet18(pretrained=True)
          my_weight = self.arch.conv1.weight.mean(dim=1, keepdim=True)    
          self.arch.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          self.arch.conv1.weight = torch.nn.Parameter(my_weight)
          num_ftrs = self.arch.fc.in_features
          self.arch.fc = nn.Linear(num_ftrs,193,bias=True)
        elif name == 'resnet34':
          self.arch = models.resnet34(pretrained=True)
          num_ftrs = self.arch.fc.in_features
          self.arch.fc = nn.Linear(num_ftrs,193,bias=True)
        elif name == 'resnext101':
          self.arch = models.resnext101_32x8d(pretrained=True)
          my_weight = self.arch.conv1.weight.mean(dim=1, keepdim=True)    
          self.arch.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          self.arch.conv1.weight = torch.nn.Parameter(my_weight)
          num_ftrs = self.arch.fc.in_features
          self.arch.fc = nn.Linear(num_ftrs,193,bias=True)
        elif name == 'resnext50':
          self.arch = models.resnext50_32x4d(pretrained=True)
          my_weight = self.arch.conv1.weight.mean(dim=1, keepdim=True)    
          self.arch.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          self.arch.conv1.weight = torch.nn.Parameter(my_weight)
          num_ftrs = self.arch.fc.in_features
          self.arch.fc = nn.Linear(num_ftrs,193,bias=True)
        elif name =='rexnetv1':
            model = rexnetv1.ReXNetV1(width_mult=1.0)
            model.output.conv2D = nn.Conv2d(1280, 1, kernel_size=(1, 1), stride=(1, 1))
          
    def forward(self, x):
        """
        """
        #x = self.convert_3_channels(x)
        x = self.arch(x)
        return x

# Predicting

In [19]:
HEIGHT = 128
WIDTH = 600
def get_transforms(): 
  train_transform = albumentations.Compose([
      #albumentations.PadIfNeeded(HEIGHT,WIDTH,border_mode = cv2.BORDER_CONSTANT,value=0),
      albumentations.Resize(HEIGHT,WIDTH),
      #albumentations.Lambda(NM(),always_apply=True)
      #Lambda(image=SpecAugment(num_mask=2,freq_masking=0.1,time_masking=0.1),mask=None,p=0.2),
      #Lambda(image=GaussNoise(2),mask=None,p=0.2),
      #albumentations.Lambda(image=CONVERTRGB(),always_apply=True),
      #albumentations.CenterCrop(100,140,p=1)
      #albumentations.RandomCrop(120,120)
      #albumentations.VerticalFlip(p=0.2),
      #albumentations.HorizontalFlip(p=0.2),
      #albumentations.RandomContrast(p=0.2),
      
      #AT.ToTensor()
      ])
  val_transform = albumentations.Compose([
      #albumentations.PadIfNeeded(HEIGHT,WIDTH,border_mode = cv2.BORDER_CONSTANT,value=0),
      albumentations.Resize(HEIGHT,WIDTH),
      #albumentations.Lambda(NM(),always_apply=True)
      #albumentations.Lambda(image=CONVERTRGB(),always_apply=True),
      #AT.ToTensor()
      ])
  return train_transform,val_transform

In [20]:
%%time
if LOAD_TEST_DATA is None:
  gc.collect()
  test = pd.read_csv('SampleSubmission.csv')
  #takes 5 minutes
  test_images = get_data(test,mode='test')
  LOAD_TEST_DATA = True
else:
  print('Data Already Loaded')


CPU times: user 242 ms, sys: 82.1 ms, total: 324 ms
Wall time: 3min 25s


In [21]:
_,test_transform = get_transforms()

In [22]:
test_dataset = ImageDataset(test_images,labels_list=None,transform=test_transform)
test_data_loader = DataLoader(dataset=test_dataset,shuffle=False,batch_size=32)

## KFOLDS

In [27]:
NFOLDS = 10
NAME = 'resnext101'
all_outputs = []
device = torch.device("cuda")
for i in range(NFOLDS):
  best_model = Net(NAME)
  #best_model.load_state_dict(torch.load(f'/content/drive/MyDrive/Resnext101GIZ/best_model_{i}'))
  best_model.load_state_dict(torch.load(f'best_model_{i}'))
  best_model = best_model.to(device)
  best_model.eval()
  fold_outputs = []
  with torch.no_grad():
    tk0 = tqdm(test_data_loader, total=len(test_data_loader))
    for bi,d in enumerate(tk0):
      images = d['image']
      #send them to device 
      images = images.to(device,dtype=torch.float)
      outputs  = best_model(images.unsqueeze(dim=1))
      outputs = torch.nn.functional.softmax(outputs)
      fold_outputs.extend(outputs.cpu().detach().numpy())
  all_outputs.append(fold_outputs)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [29]:
import scipy 
from scipy.stats.mstats import gmean

In [30]:
ss = pd.read_csv('/content/SampleSubmission.csv')
ss.loc[:,1:] = gmean(all_outputs,axis=0)
ss.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  


,fn,maize streak virus,disease,okukkoola,muwogo,mpeke,mucungwa,greens,garden,mango,bulimi,obuwuka,ebikoola,obulimi,ebisoolisooli,kaamulali,eddagala,beans,omuyembe,leaf,kisaanyi,leaves,butterfly,okuzifuuyira,micungwa,ppaapaali,emboga,kikolo,harvest,olusuku,coffee,super grow,rice,ensujju,okulima,worm,ebbugga,onion,ensigo,plantation,...,ejjobyo,omulimi,okusimba,sweet potatoes,okufuuyira,farming instructor,nnasale beedi,passion fruit,ekitooke,ebisaanyi,ekyeya,enva endiirwa,emisiri,emiyembe,amatooke,ebiwuka,farm,ebinyebwa,amappapaali,ebimera,kassooli,harvesting,emmwanyi,akamonde,obumonde,cabbages,akasaanyi,spread,ebirime,drought,kasaanyi,suckers,insects,fertilizer,nakavundira,ekiwojjolo,akawuka,ddagala,ebiwojjolo,obutungulu
0,audio_files/00118N3.wav,8.995010e-05,1.906640e-06,4.271822e-06,2.913512e-07,2.108136e-07,2.307754e-06,1.504841e-04,5.266020e-04,1.418324e-04,7.243926e-08,6.306171e-06,4.576687e-07,3.305212e-07,1.143774e-07,4.567393e-06,4.299261e-04,1.681212e-04,1.325853e-06,4.042741e-06,1.919054e-06,5.552479e-08,4.387141e-02,8.710574e-07,8.112646e-07,1.345384e-06,4.581355e-05,5.642939e-07,3.089936e-05,1.795267e-07,1.765256e-02,3.884327e-04,2.471938e-04,7.318963e-08,4.822408e-06,8.148551e-06,1.719675e-05,3.228665e-05,1.255225e-07,1.457057e-04,...,4.359684e-06,2.715650e-07,3.799191e-06,4.168113e-03,5.668909e-06,7.773392e-06,6.482424e-08,3.875051e-04,1.240526e-07,7.601890e-07,1.562818e-07,2.089796e-07,5.050547e-09,2.979610e-07,3.716720e-07,1.065892e-05,2.425141e-06,1.925404e-06,2.504007e-04,5.989808e-07,1.989752e-06,3.522669e-06,3.055035e-07,5.534263e-06,2.170022e-06,4.225230e-03,1.621191e-04,1.643851e-04,8.199180e-08,2.837460e-06,5.130774e-06,1.737540e-05,1.459880e-06,4.074054e-05,4.155475e-06,3.561648e-06,1.792034e-05,2.820633e-04,1.559326e-06,1.342427e-06
1,audio_files/00P0NMV.wav,1.807206e-07,7.058435e-08,7.227641e-08,3.884817e-07,2.537381e-07,1.907346e-06,3.442448e-08,5.050384e-07,6.823348e-09,2.946973e-07,7.308491e-07,2.770804e-07,9.445325e-07,3.168471e-07,8.351718e-07,1.224257e-06,9.929655e-09,8.023891e-07,6.824380e-08,2.655529e-07,1.378820e-08,1.198171e-06,9.385009e-06,7.505004e-07,2.306701e-07,5.535542e-07,6.869985e-08,1.274321e-07,4.195062e-08,5.553825e-08,2.251009e-07,9.487131e-09,1.014823e-06,5.384258e-07,2.464546e-08,1.552911e-06,2.314702e-08,9.517077e-08,1.427378e-06,...,8.930194e-07,5.064501e-07,7.050029e-07,1.274166e-07,1.446972e-05,4.395811e-07,1.729133e-05,4.066411e-07,1.364431e-07,5.336858e-07,7.267234e-07,1.895565e-05,2.954182e-07,4.972502e-07,1.139401e-06,1.584955e-06,4.458061e-09,8.672271e-07,8.649611e-06,2.452455e-06,8.744091e-07,3.113621e-07,1.585713e-07,2.464881e-04,3.007171e-06,2.898930e-07,6.094816e-06,1.415826e-08,2.652600e-07,5.447848e-08,2.914673e-07,1.138339e-07,6.682754e-08,3.261368e-06,9.985391e-01,9.891675e-06,1.284848e-05,1.254168e-05,3.988020e-06,7.360574e-07
2,audio_files/01QEEZI.wav,5.841715e-07,1.699345e-06,2.800538e-06,2.144980e-04,3.660691e-06,9.136203e-06,1.878193e-06,2.030640e-06,4.438761e-08,2.753188e-05,5.367719e-05,2.077933e-06,5.866117e-05,1.606600e-06,1.423407e-05,1.159439e-06,2.270500e-07,9.007055e-01,2.149107e-06,3.719460e-06,1.571889e-06,3.017010e-07,1.388138e-05,3.518831e-06,1.510866e-06,2.440156e-06,2.934903e-06,2.959410e-07,8.846298e-06,2.153747e-07,5.052426e-07,2.119845e-07,1.287526e-06,3.772801e-06,5.222025e-06,4.582778e-06,1.378383e-07,1.681393e-06,6.763389e-07,...,4.550793e-06,1.909778e-05,1.928728e-05,3.717068e-07,8.946284e-06,2.095504e-07,4.572797e-06,3.448664e-07,7.144434e-07,1.092249e-06,4.383481e-06,2.089886e-05,4.455654e-06,2.226120e-02,1.486332e-06,3.520288e-06,1.155641e-07,2.977904e-04,3.048988e-06,5.109353e-06,1.092534e-05,9.606024e-07,2.340988e-05,7.729961e-06,3.214384e-04,2.622332e-07,7.180730e-06,2.464804e-07,4.194643e-06,1.989056e-06,3.201061e-06,2.005753e-07,1.747311e-07,1.079271e-06,3.411264e-06,2.098129e-06,2.197728e-06,4.324486e-06,6.939806e-06,1.334599e-05
3,audio_files/037YAED.wav,1.010792e-06,2.296856e-07,1.114050e-07,5.200289e-07,7.947726e-05,7.824970e

In [32]:
ss1 = pd.read_csv('/content/SampleSubmission.csv')
ss1.loc[:,1:] = np.mean(all_outputs,axis=0)
ss1.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  


,fn,maize streak virus,disease,okukkoola,muwogo,mpeke,mucungwa,greens,garden,mango,bulimi,obuwuka,ebikoola,obulimi,ebisoolisooli,kaamulali,eddagala,beans,omuyembe,leaf,kisaanyi,leaves,butterfly,okuzifuuyira,micungwa,ppaapaali,emboga,kikolo,harvest,olusuku,coffee,super grow,rice,ensujju,okulima,worm,ebbugga,onion,ensigo,plantation,...,ejjobyo,omulimi,okusimba,sweet potatoes,okufuuyira,farming instructor,nnasale beedi,passion fruit,ekitooke,ebisaanyi,ekyeya,enva endiirwa,emisiri,emiyembe,amatooke,ebiwuka,farm,ebinyebwa,amappapaali,ebimera,kassooli,harvesting,emmwanyi,akamonde,obumonde,cabbages,akasaanyi,spread,ebirime,drought,kasaanyi,suckers,insects,fertilizer,nakavundira,ekiwojjolo,akawuka,ddagala,ebiwojjolo,obutungulu
0,audio_files/00118N3.wav,1.274808e-03,4.204511e-05,6.237684e-05,4.306555e-06,2.814463e-06,1.607619e-05,1.197155e-03,2.718421e-03,1.199887e-02,7.294782e-07,4.813337e-05,4.115482e-06,2.311262e-06,3.832717e-06,6.096682e-05,2.504772e-03,1.077231e-03,1.557851e-05,7.340847e-05,6.897688e-06,4.283868e-06,1.688640e-01,4.331717e-06,1.166909e-05,2.152204e-05,1.420331e-03,2.377288e-06,1.068861e-03,1.486548e-06,1.011970e-01,1.092834e-02,1.169839e-03,2.204750e-06,3.959886e-05,3.192803e-04,0.000109,3.624743e-04,2.481721e-06,1.275324e-03,...,0.000020,3.469648e-06,4.055526e-05,9.321040e-03,0.000028,5.007892e-05,3.340939e-06,5.212928e-04,2.124851e-06,2.436970e-06,6.347234e-07,0.000007,1.905683e-07,6.204203e-06,3.855152e-05,0.000030,3.280520e-05,9.574740e-06,1.161666e-02,3.597760e-06,5.258819e-06,1.200118e-04,2.837678e-06,3.171012e-05,0.000034,6.981567e-02,2.898964e-03,6.381084e-03,7.233490e-07,1.227630e-04,3.208090e-05,4.793935e-04,3.389419e-05,6.486089e-04,0.000021,2.048837e-05,3.415250e-04,4.812017e-04,0.000027,8.556877e-06
1,audio_files/00P0NMV.wav,3.919516e-07,2.261279e-07,3.476127e-07,5.375873e-07,3.800099e-07,3.464019e-06,5.609616e-08,9.467680e-07,1.753025e-08,6.816296e-07,1.401712e-06,7.911262e-07,2.009571e-06,6.559958e-07,2.117675e-06,2.364973e-06,2.006250e-08,1.689022e-06,3.552123e-07,4.284959e-07,2.594807e-08,2.543610e-06,1.817512e-05,1.291136e-06,3.590616e-07,1.328434e-06,1.081773e-07,1.683885e-07,6.354847e-08,8.344541e-08,3.695749e-07,1.867378e-08,1.582495e-06,1.478676e-06,4.487374e-08,0.000003,6.712948e-08,1.394039e-07,3.620663e-06,...,0.000002,7.530092e-07,1.324124e-06,2.058181e-07,0.000018,5.112155e-07,2.628982e-05,5.962641e-07,2.893154e-07,6.444237e-07,1.378152e-06,0.000055,4.251820e-07,8.456600e-07,3.087127e-06,0.000003,1.160316e-08,1.480670e-06,1.658915e-05,9.784376e-06,2.024096e-06,7.340910e-07,3.155495e-07,3.765374e-04,0.000007,1.321313e-06,1.219722e-05,2.538391e-08,4.524194e-07,7.002755e-08,3.053344e-07,2.398247e-07,8.945717e-08,7.790199e-06,0.998540,1.684586e-05,1.739345e-05,2.377134e-05,0.000010,1.437332e-06
2,audio_files/01QEEZI.wav,1.392452e-06,1.309995e-05,9.899577e-06,1.780332e-02,1.235830e-05,2.148051e-05,7.971152e-06,8.108736e-06,5.510450e-07,1.403641e-04,7.124472e-04,9.283902e-06,3.678731e-04,7.182837e-06,2.647544e-05,3.047541e-06,1.230522e-06,9.066128e-01,4.751882e-05,6.117870e-06,7.381364e-05,1.009775e-06,4.505112e-05,8.169993e-06,4.081503e-06,1.018388e-05,6.978781e-06,8.384712e-07,2.853698e-05,8.032206e-07,3.076083e-06,1.251249e-06,5.270949e-06,1.170544e-05,1.898827e-05,0.000018,7.508747e-07,5.295921e-06,2.342601e-06,...,0.000017,6.006738e-05,7.598905e-05,1.350609e-06,0.000020,7.634754e-07,1.901343e-05,2.000082e-06,1.841452e-06,5.606488e-06,1.118521e-05,0.000055,2.089020e-05,3.576116e-02,8.442154e-06,0.000015,3.264162e-07,1.451119e-03,8.840278e-06,5.158851e-05,2.486913e-05,2.669732e-06,7.301329e-05,1.791159e-05,0.001791,8.834133e-07,1.729162e-05,5.326480e-07,3.574687e-05,4.000998e-06,8.296038e-06,7.684666e-07,8.550284e-07,1.020087e-05,0.000010,5.922974e-06,8.607427e-06,9.917863e-06,0.000103,4.328849e-05
3,audio_files/037YAED.wav,1.807516e-05,5.574678e-06,6.737023e-06,5.128734e-06,4.578517e-03,5.681848e-05,1.896399e-06,7.461973e-01,3.626902e-07,9.884436e-07,1.601151e-05,2.838325e-06,3.721509e-07,7

In [34]:
ss.iloc[:,1:] = (ss1.iloc[:,1:] + ss.iloc[:,1:])/2
ss.head()


,fn,maize streak virus,disease,okukkoola,muwogo,mpeke,mucungwa,greens,garden,mango,bulimi,obuwuka,ebikoola,obulimi,ebisoolisooli,kaamulali,eddagala,beans,omuyembe,leaf,kisaanyi,leaves,butterfly,okuzifuuyira,micungwa,ppaapaali,emboga,kikolo,harvest,olusuku,coffee,super grow,rice,ensujju,okulima,worm,ebbugga,onion,ensigo,plantation,...,ejjobyo,omulimi,okusimba,sweet potatoes,okufuuyira,farming instructor,nnasale beedi,passion fruit,ekitooke,ebisaanyi,ekyeya,enva endiirwa,emisiri,emiyembe,amatooke,ebiwuka,farm,ebinyebwa,amappapaali,ebimera,kassooli,harvesting,emmwanyi,akamonde,obumonde,cabbages,akasaanyi,spread,ebirime,drought,kasaanyi,suckers,insects,fertilizer,nakavundira,ekiwojjolo,akawuka,ddagala,ebiwojjolo,obutungulu
0,audio_files/00118N3.wav,6.823789e-04,2.197588e-05,3.332433e-05,2.298953e-06,1.512638e-06,9.191974e-06,6.738193e-04,1.622512e-03,6.070353e-03,4.009588e-07,2.721977e-05,2.286575e-06,1.320892e-06,1.973547e-06,3.276711e-05,1.467349e-03,6.226760e-04,8.452179e-06,3.872560e-05,4.408371e-06,2.169696e-06,1.063677e-01,2.601387e-06,6.240176e-06,1.143371e-05,7.330725e-04,1.470791e-06,5.498804e-04,8.330374e-07,5.942478e-02,5.658387e-03,7.085163e-04,1.138970e-06,2.221063e-05,1.637144e-04,0.000063,1.973805e-04,1.303622e-06,7.105151e-04,...,1.203836e-05,1.870607e-06,2.217722e-05,6.744577e-03,1.664767e-05,2.892616e-05,1.702882e-06,4.543989e-04,1.124452e-06,1.598579e-06,3.955026e-07,0.000003,9.780941e-08,3.251082e-06,1.946159e-05,0.000020,1.761517e-05,5.750072e-06,5.933531e-03,2.098370e-06,3.624285e-06,6.176723e-05,1.571591e-06,1.862219e-05,1.811823e-05,3.702045e-02,1.530541e-03,3.272735e-03,4.026704e-07,6.280023e-05,1.860584e-05,2.483844e-04,1.767703e-05,3.446747e-04,0.000012,1.202501e-05,1.797227e-04,3.816325e-04,0.000014,4.949652e-06
1,audio_files/00P0NMV.wav,2.863361e-07,1.483561e-07,2.099446e-07,4.630345e-07,3.168740e-07,2.685682e-06,4.526032e-08,7.259032e-07,1.217680e-08,4.881634e-07,1.066280e-06,5.341033e-07,1.477052e-06,4.864215e-07,1.476424e-06,1.794615e-06,1.499608e-08,1.245706e-06,2.117281e-07,3.470244e-07,1.986813e-08,1.870891e-06,1.378006e-05,1.020818e-06,2.948659e-07,9.409942e-07,8.843856e-08,1.479103e-07,5.274955e-08,6.949183e-08,2.973379e-07,1.408045e-08,1.298659e-06,1.008551e-06,3.475960e-08,0.000002,4.513825e-08,1.172873e-07,2.524021e-06,...,1.350676e-06,6.297296e-07,1.014564e-06,1.666173e-07,1.604694e-05,4.753983e-07,2.179057e-05,5.014526e-07,2.128793e-07,5.890548e-07,1.052437e-06,0.000037,3.603001e-07,6.714551e-07,2.113264e-06,0.000002,8.030609e-09,1.173949e-06,1.261938e-05,6.118416e-06,1.449253e-06,5.227266e-07,2.370604e-07,3.115127e-04,5.183379e-06,8.056030e-07,9.146019e-06,1.977108e-08,3.588397e-07,6.225302e-08,2.984008e-07,1.768293e-07,7.814236e-08,5.525784e-06,0.998539,1.336877e-05,1.512097e-05,1.815651e-05,0.000007,1.086695e-06
2,audio_files/01QEEZI.wav,9.883119e-07,7.399647e-06,6.350058e-06,9.008907e-03,8.009495e-06,1.530836e-05,4.924672e-06,5.069688e-06,2.977163e-07,8.394801e-05,3.830622e-04,5.680917e-06,2.132671e-04,4.394718e-06,2.035476e-05,2.103490e-06,7.287860e-07,9.036591e-01,2.483397e-05,4.918665e-06,3.769276e-05,6.557378e-07,2.946625e-05,5.844412e-06,2.796185e-06,6.312016e-06,4.956842e-06,5.672061e-07,1.869164e-05,5.092977e-07,1.790663e-06,7.316169e-07,3.279238e-06,7.739121e-06,1.210515e-05,0.000011,4.443565e-07,3.488657e-06,1.509470e-06,...,1.057128e-05,3.958258e-05,4.763816e-05,8.611581e-07,1.433113e-05,4.865129e-07,1.179312e-05,1.172474e-06,1.277948e-06,3.349368e-06,7.784344e-06,0.000038,1.267293e-05,2.901118e-02,4.964243e-06,0.000010,2.209902e-07,8.744548e-04,5.944633e-06,2.834893e-05,1.789723e-05,1.815167e-06,4.821158e-05,1.282078e-05,1.056372e-03,5.728232e-07,1.223618e-05,3.895642e-07,1.997075e-05,2.995027e-06,5.748550e-06,4.845210e-07,5.148798e-07,5.640071e-06,0.000007,4.010551e-06,5.402577e-06,7.121175e-06,0.000055,2.831724e-05
3,audio_files/037YAED.wav,9.542977e-06,2.902182e-06,3.424214e-06,2.824381e-06,2.328997e-03,2.880049e-05,1.003178e-06,6.425138e-01,1.897131e-07,5.763852e-07,8.28

In [35]:
ss.to_csv(f'resnext101_mean_gmean.csv',index=False)